In [3]:
from app import Manga, MangaChapter, db

db.create_all()  

# Ajouter les manga à la table de base
one_piece = Manga(name = "One Piece",url = "https://www.scan-vf.net/one_piece/", cover="")
boruto = Manga(name = "Boruto", url= "https://www.scan-vf.net/boruto", cover="")
fairytail100 = Manga(name = "Fairy Tail 100 Years Quest", url= "https://www.scan-vf.net/fairy-tail-100-years-quest", cover="")
onepunchman = Manga(name = "One Punch Man", url= "https://www.scan-vf.net/one-punch-man", cover="")
hxh = Manga(name = "Hunter X Hunter", url= "https://www.scan-vf.net/hunter-x-hunter", cover="")


db.session.add_all([one_piece,boruto,fairytail100,onepunchman,hxh]) 

db.session.commit()

PendingRollbackError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.IntegrityError) UNIQUE constraint failed: manga.name
[SQL: INSERT INTO manga (name, url, cover) VALUES (?, ?, ?)]
[parameters: ('One Piece', 'https://www.scan-vf.net/one_piece/', '')]
(Background on this error at: https://sqlalche.me/e/14/gkpj) (Background on this error at: https://sqlalche.me/e/14/7s2a)

In [2]:

# Ajout d'un chapitre à la table chapitre
# Lecture : https://www.youtube.com/watch?v=VVX7JIWx-ss
chap_1044_onepiece = MangaChapter(chapter_number = '1044', chapter_url = 'https://www.scan-vf.net/one_piece/chapitre-1044/' , manga = Manga(name="one_piece" ))
db.session.add(chap_1044_onepiece)
db.session.commit()

IntegrityError: (sqlite3.IntegrityError) NOT NULL constraint failed: manga.url
[SQL: INSERT INTO manga (name, url, cover) VALUES (?, ?, ?)]
[parameters: ('one_piece', None, None)]
(Background on this error at: https://sqlalche.me/e/14/gkpj)